# Scrabu Prototype (for a List of Shipment Numbers)

This notebook is for creating a prototype of the Scrabu project. The goal is to download DHL pages for a specific shipment number and scrap the shipment information from it.

#### Generate a list of shipment numbers

In [ ]:
def generate_shipment_numbers(shipment_number=340434188193324407, size=100):
    shipment_numbers_list = []
    for i in range(1, size):
        shipment_numbers = list(map(int, str(shipment_number)))
        shipment_number = str(shipment_number).rjust(20, '0')
        shipment_numbers_list.append(shipment_number)
    return shipment_numbers_list

In [ ]:
shipment_numbers = generate_shipment_numbers(shipment_number=340434188193324407, size=100)
print("First elements of the list", shipment_numbers[0:3])

#### Download the HTML content for a specific shipment number

In [ ]:
def request(shipment_number="340434188193324407", start_url="https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode="):
    import requests
    import numpy as np
    request_url = start_url + shipment_number
    return requests.get(request_url).content

In [ ]:
html_response = request(shipment_number="340434188193324407")
print("Sample of HTML response:\n", html_response[0:100])

#### Parsing HTML and converting it into JSON

In [ ]:
def html_to_json(html, start_url="https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode=", shipment_number=None):
    from lxml import etree
    import json
    
    html_tree = etree.HTML(html)
    
    def clean_json(json):
        start = json.find('JSON.parse(')
        end = json.find('"),', start)
        json = json[start:end]
        json = json.replace('JSON.parse("', '')
        json = json.replace('\\', '')
        return json
    
    def find_json_element(html_tree):
        json_element = html_tree.xpath('//div')
        return str(etree.tostring(json_element[0]))
    
    dirty_json = find_json_element(html_tree)
    json_string = clean_json(dirty_json)
    return json.loads(json_string)

In [ ]:
shipment_details_json = html_to_json(html_response)
print(shipment_details_json)

#### Structure the shipment details in JSON format

In [ ]:
def shipment_details(shipment_details_json, start_url="https://www.dhl.de/int-verfolgen/search?language=de&lang=de&domain=de&piececode=", shipment_number="00340434188193324407"):
    import datetime
    delivery_history_dict = {}
    delivery_history_dict["shipment_number"] = shipment_details_json["sendungen"][0]["sendungsdetails"]["sendungsnummern"].get("sendungsnummer")
    delivery_history_dict["crawltime"] = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    delivery_history_dict["url"] = start_url + shipment_number 
    delivery_history_dict["events"] = shipment_details_json["sendungen"][0]["sendungsdetails"]["sendungsverlauf"].get("events", [])
    return delivery_history_dict

In [ ]:
shipment_history = shipment_details(shipment_details_json)
print(shipment_history)

In [ ]:
import json
def save_dictionary(shipment_history=None, filename=None):
    print("Writing file {}".format(filename))
    with open(filename, 'w') as f:
        json.dump(shipment_history, f)

In [ ]:
save_dictionary(shipment_history, filename="../data/00340434188193324407.json")

#### Main function, which goes through all shipment numbers and uses the previous methods

In [ ]:
import time
def main():
    shipment_numbers = generate_shipment_numbers(shipment_number=340434188193324407, size=100)
    for shipment_number in shipment_numbers:
        time.sleep(0.5)
        print("Processing shipment number {}".format(shipment_number))
        html_response = request(shipment_number=shipment_number)
        shipment_details_json = html_to_json(html_response)
        shipment_history = shipment_details(shipment_details_json)
        save_dictionary(shipment_history, filename="../data/{}.json".format(shipment_number))

In [ ]:
main()